# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

Признаки
- <s>DateCrawled — дата скачивания анкеты из базы</s>
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет (в оригинальном датасете описывается как notRepairedDamage - имеются ли не восстановленные повреждения)
- <s>DateCreated — дата создания анкеты</s>
- NumberOfPictures — количество фотографий автомобиля
- <s>PostalCode — почтовый индекс владельца анкеты (пользователя)
- <s>LastSeen — дата последней активности пользователя

Целевой признак
- Price — цена (евро)

In [38]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler

from lightgbm import LGBMRegressor
from catboost import Pool, CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV 
from sklearn.metrics import mean_squared_error


Обновление sklearn для использования метрики *neg_root_mean_squared_error*. После выполнения потребуется перезапустить ядро

In [39]:
#pip install -U scikit-learn

## Подготовка данных

In [40]:
try:
    raw_data = pd.read_csv('/datasets/autos.csv', parse_dates=[0, 12])
except:
    raw_data = pd.read_csv('autos.csv', parse_dates=[0, 12])
df = raw_data.copy()

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null datetime64[ns]
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null datetime64[ns]
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: datetime64[ns](2), int64(7), object(7)
memory usage: 43.3+ MB


In [42]:
df['YearCrawled'] = df['DateCrawled'].dt.year
df['MonthCrawled'] = df['DateCrawled'].dt.month

In [43]:
df = df[(df['YearCrawled']>df['RegistrationYear'])|((df['YearCrawled']==df['RegistrationYear'])&(df['MonthCrawled']>df['RegistrationMonth']))]

In [44]:
df.describe(percentiles=[0.01,0.05, 0.1, 0.25,0.50,0.75,0.95,0.99])

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode,YearCrawled,MonthCrawled
count,333932.000000,333932.000000,333932.000000,333932.000000,333932.000000,333932.0,333932.000000,333932.0,333932.000000
mean,4506.958923,2002.080924,111.442470,127963.971707,5.729337,0.0,50656.495484,2016.0,3.162626
std,4557.786578,13.949480,183.826722,37960.400025,3.717611,0.0,25813.142319,0.0,0.369025
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000,2016.0,3.000000
1%,0.000000,1979.000000,0.000000,5000.000000,0.000000,0.0,2694.000000,2016.0,3.000000
5%,200.000000,1992.000000,0.000000,40000.000000,0.000000,0.0,9573.000000,2016.0,3.000000
10%,500.000000,1995.000000,0.000000,70000.000000,0.000000,0.0,15230.000000,2016.0,3.000000
25%,1100.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30177.000000,2016.0,3.000000
50%,2800.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49525.000000,2016.0,3.000000
75%,6500.000000,2007.000000,143.000000,150000.000000,9.000000,0.0,71263.000000,2016.0,3.000000


- Нам более не нужны колонки относящиеся ко времени извлечения объявления, также видим, что все объявления без фотографий - можно избавиться от соответствующей колонки, и уберём *PostalCode*
- *RegistrationMonth* для многих объявлений - 0. Скорее всего от данной колонки тоже можно избавиться, время выпуска для машин важно с точностью до года, а не месяца
- Минимальное значение *RegistrationYear* -1000, но таких значений крайне мало, 99% больше 1979. Что-то похожее на автомобиль впервые появилось в 1863, и одна из первых машин для широкой публики в 1908. Стоимость Ford Model T в данный момент колеблется от 17000 до 25000USD, Видим что максимальная цена в нашем наборе - 20000EUR. Так что уберём все объявления с *RegistrationYear*<1908
- Некоторое количество объявлений с неуказанной ценой - нужно удалить.
- В колонке *Power* подозрительны значения меньше 20 (мощность первого форда) и максимум в 20000. Но, как видим, 99% меньше 301лс, Сверху ограничим мощность 1000 и снизу 20.


In [45]:
df = df.drop(columns=['PostalCode', 'DateCrawled', 'YearCrawled', 'MonthCrawled', 'NumberOfPictures', 'DateCreated', 'LastSeen', 'RegistrationMonth' ])

In [46]:
df =df[(df['RegistrationYear']>=1908) 
      &(df['Price']>0)
      &(df['Power']>=20)
      &(df['Power']<=1000)]

In [47]:
def missing_values_percentage(df):
    count=round(df.isnull().sum(),2)
    percent=round((df.isnull().sum()/df.shape[0])*100,2)
    data=pd.concat([count,percent],axis=1)
    data.reset_index(inplace=True)
    data.rename(columns={0: 'Missing Values Count',1: 'Missing Values %'},inplace=True)
    missing = data[data['Missing Values Count']!=0].sort_values(by = 'Missing Values %', ascending = False)
    display(missing)
    return missing

In [48]:
missing = missing_values_percentage(df)

,index,Missing Values Count,Missing Values %
9,NotRepaired,41246,14.13
7,FuelType,13731,4.70
5,Model,10506,3.60
1,VehicleType,6844,2.34
3,Gearbox,5145,1.76


In [49]:
for item in list(missing['index'].values):
    display(df[item].value_counts())
    print('\n')

no     222345
yes     28400
Name: NotRepaired, dtype: int64

petrol      185622
gasoline     87189
lpg           4647
cng            477
hybrid         199
other           84
electric        42
Name: FuelType, dtype: int64

golf                  24193
other                 20230
3er                   17241
polo                  10471
corsa                  9930
                      ...  
kalina                    5
serie_2                   5
serie_3                   3
range_rover_evoque        2
rangerover                2
Name: Model, Length: 249, dtype: int64

sedan          83049
small          69906
wagon          59255
bus            26201
convertible    18752
coupe          14604
suv            10951
other           2429
Name: VehicleType, dtype: int64

manual    228565
auto       58281
Name: Gearbox, dtype: int64

В колонке *Model* 249 различных значений, OneHotEncoding добавит к данным 248 столбцов, а порядковое кодирование может неверно трактоваться некоторыми моделями, поэтому её мы удалим 
Также удалим пропуски с долей менее 10%, а пропуски в NotRepaired заменим на NA
Затем применим OHE к оставшимся категориальным переменным

In [50]:
df_clean = df.drop(columns='Model')
df_clean = df_clean.dropna(axis = 0, subset = ['Gearbox', 'VehicleType', 'FuelType'])

In [51]:
clean_missing = missing_values_percentage(df_clean)

,index,Missing Values Count,Missing Values %
8,NotRepaired,32536,12.01


In [52]:
df_clean.fillna('NA', inplace=True)

Для последующей работы с алгоритмами CatBoost и LightGBM преобразуем категориальные признаки в формат *category*

In [53]:
df_clean[['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'NotRepaired']]= df_clean[['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'NotRepaired']].astype('category')

In [54]:
df_encoded = pd.get_dummies(df_clean, drop_first='true', prefix_sep='_')

In [55]:
df_encoded

,Price,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no,NotRepaired_yes
1,18300,2011,190,125000,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1500,2001,75,150000,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
4,3600,2008,69,90000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5,650,1995,102,150000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354359,7900,2010,140,150000,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
354362,3200,2004,225,150000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
354366,1199,2000,101,125000,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
354367,9200,1996,102,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


Данные подготовлены, далее делим на выборки для обучения и тестирования и стандартизируем признаки

In [56]:
X_train, X_test, y_train, y_test = train_test_split(df_encoded.drop(columns=['Price']),
                                                    df_encoded['Price'],
                                                    test_size=0.25,
                                                    random_state=42)

X_train_wc, X_test_wc, y_train_wc, y_test_wc = train_test_split(df_clean.drop(columns=['Price']),
                                                    df_clean['Price'],
                                                    test_size=0.25,
                                                    random_state=42)

In [57]:
numeric = ['RegistrationYear', 'Power', 'Kilometer']
scaler = StandardScaler()
scaler.fit(X_train[numeric])
for data in [X_train, X_test, X_train_wc, X_test_wc]:
    data[numeric] = scaler.transform(data[numeric])
    display(data.head(2))

,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no,NotRepaired_yes
224612,-0.112835,-1.414979,-1.035043,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
71575,0.690944,0.125847,0.599552,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0


,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no,NotRepaired_yes
343781,-0.434347,1.299809,0.599552,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
102412,-0.595103,-0.130958,0.599552,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


,VehicleType,RegistrationYear,Gearbox,Power,Kilometer,FuelType,Brand,NotRepaired
224612,small,-0.112835,auto,-1.414979,-1.035043,petrol,smart,no
71575,wagon,0.690944,manual,0.125847,0.599552,petrol,toyota,no


,VehicleType,RegistrationYear,Gearbox,Power,Kilometer,FuelType,Brand,NotRepaired
343781,sedan,-0.434347,auto,1.299809,0.599552,petrol,volkswagen,no
102412,bus,-0.595103,manual,-0.130958,0.599552,petrol,opel,no


## Обучение моделей

Для обучения и проверки нескольких моделей с различными параметрами применим RandomizedSearchCV.


In [58]:
search_space_boost_models = [{'regressor': [LGBMRegressor(random_state=42, n_jobs=-1, verbose=0)],
                              'regressor__n_estimators': [100, 200],
                              'regressor__max_depth': [10, 20]},

                             {'regressor': [CatBoostRegressor(random_state=42,
                                                loss_function='RMSE',
                                                verbose = 0,
                                                cat_features=['VehicleType',
                                                              'Gearbox',
                                                              'FuelType',
                                                              'Brand',
                                                              'NotRepaired'])],
                              'regressor__iterations': [100, 200],
                              'regressor__l2_leaf_reg': np.logspace(-2, 2, 5)}]

kfold = KFold(n_splits = 3, random_state=42, shuffle=True)

pipeline = Pipeline(steps = [['regressor', LinearRegression()]
                                   ])

grid_search_boost = RandomizedSearchCV(estimator=pipeline,
                                 param_distributions = search_space_boost_models,
                                 scoring='neg_root_mean_squared_error',
                                 cv=kfold,
                                 verbose = 0,
                                 n_iter=8,
                                 n_jobs=-1,
                                random_state = 42)

In [ ]:
best_model_boost = grid_search_boost.fit(X_train_wc, y_train_wc)

In [67]:
search_space = [{'regressor': [RandomForestRegressor(random_state=42, n_jobs=-1, verbose=0)],
                'regressor__n_estimators': [50, 100, 200],
                'regressor__max_depth': [10, 15, 20]},
                
                {'regressor': [LinearRegression()]}]

grid_search_common = RandomizedSearchCV(estimator=pipeline,
                                 param_distributions = search_space,
                                 scoring='neg_root_mean_squared_error',
                                 cv=kfold,
                                 verbose = 0,
                                 n_iter=8,
                                 n_jobs=-1,
                                random_state = 42)

In [68]:
best_model_common = grid_search_common.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END regressor=RandomForestRegressor(n_jobs=-1, random_state=42), regressor__max_depth=20, regressor__n_estimators=200; total time= 2.0min
[CV] END regressor=RandomForestRegressor(n_jobs=-1, random_state=42), regressor__max_depth=20, regressor__n_estimators=200; total time= 2.1min
[CV] END regressor=RandomForestRegressor(n_jobs=-1, random_state=42), regressor__max_depth=20, regressor__n_estimators=200; total time= 2.0min
[CV] END regressor=RandomForestRegressor(n_jobs=-1, random_state=42), regressor__max_depth=10, regressor__n_estimators=100; total time=  40.0s
[CV] END regressor=RandomForestRegressor(n_jobs=-1, random_state=42), regressor__max_depth=10, regressor__n_estimators=100; total time=  38.9s
[CV] END regressor=RandomForestRegressor(n_jobs=-1, random_state=42), regressor__max_depth=10, regressor__n_estimators=100; total time=  39.6s
[CV] END regressor=RandomForestRegressor(n_jobs=-1, random_state=42), regressor__m

Для лучшей модели выводим метрику, посчитанную на тестовой выборке

In [70]:
print('Лучшая RMSE для бустинг-моделей:',abs(best_model_boost.score(X_test_wc, y_test_wc)))

итоговая RMSE для бустинг-моделей: 1629.7049270073749


In [71]:
print('Лучшая RMSE для стандартных моделей:',abs(best_model_common.score(X_test, y_test)))

итоговая RMSE для стандартных моделей: 1606.3681791860554


Создадим таблицы результатов с очками и средним временем, затрачиваемым на обучение и оценку

In [88]:
cv_results_boost = pd.DataFrame(grid_search_boost.cv_results_)[['mean_test_score', 'mean_fit_time', 'mean_score_time', 'param_regressor']]

In [89]:
cv_results_common = pd.DataFrame(grid_search_common.cv_results_)[['mean_test_score', 'mean_fit_time', 'mean_score_time', 'param_regressor']]

## Анализ моделей

Отобразим общую таблицу результатов

In [102]:
results = pd.concat([cv_results_boost, cv_results_common], axis=0).sort_values(by='mean_test_score', ascending = False).reset_index(drop=True)
results

,mean_test_score,mean_fit_time,mean_score_time,param_regressor
0,-1633.115791,116.788597,3.786553,"RandomForestRegressor(max_depth=20, n_estimato..."
1,-1635.008195,7.429345,1.098301,"LGBMRegressor(max_depth=10, n_estimators=200, ..."
2,-1636.202653,61.788870,1.967403,"RandomForestRegressor(max_depth=20, n_estimato..."
3,-1672.460632,4.689451,0.672064,"LGBMRegressor(max_depth=10, n_estimators=200, ..."
4,-1672.769730,106.301860,2.666438,"RandomForestRegressor(max_depth=20, n_estimato..."
5,-1674.508675,132.879448,0.753912,"LGBMRegressor(max_depth=10, n_estimators=200, ..."
6,-1675.082124,49.479310,1.292377,"RandomForestRegressor(max_depth=20, n_estimato..."
7,-1828.492493,51.217410,0.070902,<catboost.core.CatBoostRegressor object at 0x7...
8,-1828.726501,51.803281,0.077350,<catboost.core.CatBoostRegressor object at 0x7...
9,-1834.530564,50.015895,0.065541,<catboost.core.CatBoostRegressor object at 0x7...


Random Forest показывает лучший результат, LGBM немного отстаёт по метрике, но времени на обучение тратится на порядки меньше.
Проверим, можно ли улучшить результат регулировкой параметров

Выведем данные по LightGBM

In [97]:
cv_results_boost.iloc[7]['param_regressor']

LGBMRegressor(max_depth=10, n_estimators=200, random_state=42, verbose=0)

для GridSearchCV попробуем увеличить *max_depth* и *num_leaves* и перебором подобрать коэффициенты регуляризации

In [99]:
lgbmodel = LGBMRegressor(random_state=42, n_jobs=-1, verbose=0, n_estimators=200, max_depth = 20, num_leaves = 120)
param_grid = {
             'reg_alpha': np.logspace(-2,1, 4),
             'reg_lambda': np.logspace(-2,1,4)}
grid_search = GridSearchCV(estimator=lgbmodel,
                           param_grid= param_grid,
                           scoring='neg_root_mean_squared_error',
                           cv=kfold,
                           verbose = 1,
                           n_jobs=-1)


In [103]:
test_results = grid_search.fit(X_train_wc, y_train_wc)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


In [104]:
print('итоговая RMSE:',abs(test_results.score(X_test_wc, y_test_wc)))

итоговая RMSE: 1569.850271532248


In [105]:
pd.DataFrame(test_results.cv_results_).columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_reg_alpha', 'param_reg_lambda', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score'],
      dtype='object')

In [106]:
pd.DataFrame(test_results.cv_results_).sort_values(by = 'rank_test_score')[['rank_test_score','mean_test_score',
                                                                       'std_test_score',
                                                                       'mean_fit_time',
                                                                       'param_reg_alpha',
                                                                       'param_reg_lambda' ]].head(10)

,rank_test_score,mean_test_score,std_test_score,mean_fit_time,param_reg_alpha,param_reg_lambda
1,1,-1577.708333,7.347375,47.475673,0.01,0.1
5,2,-1577.708353,7.347377,77.476446,0.1,0.1
14,3,-1578.003388,8.569142,24.857642,10,1
0,4,-1578.143187,9.284744,2066.486974,0.01,0.01
4,5,-1578.143206,9.284740,82.418489,0.1,0.01
13,6,-1578.438314,8.207962,25.361387,10,0.1
8,7,-1578.563266,8.766087,84.430739,1,0.01
12,8,-1578.793271,8.763377,25.761909,10,0.01
9,9,-1578.917636,7.563250,26.084944,1,0.1
10,10,-1579.956133,8.672055,24.525769,1,1


Путём подбора параметров, немного превзошли результат RandomForest, сохраняя малое время обучения модели. Таким образом, LGBM оказался более эффективен

(но при использовании категориальных признаков без предварительного кодирования, в некоторых случаях время обучения резко возрастает)

## Вывод

- Входные данные содержали аномалии и отсутствующие значения и были удалены, для работы осталось 270837 строк из исходных 354369 - около 75%
- Для сравнения были выбраны модели Linear Regression и Random Forest из библиотеки sklearn и модели с использованием градиентного бустинга - CatBoost и LightGBM. Для обычных моделей категориальные признаки были преобразованы методом One Hot Encoding. Для CatBoost и LGBM категориальные признаки оставлены в изначальном виде, так как алгоритмы имеют встроенные методы работы с ними.
- Для сравнения был использован RandomizedSearchCV с перебором нескольких параметров для каждой модели. Лучшие результаты показали Random Forest и LightGBM с заметным преимуществом последнего по времени обучения и оценки
- После дополнительной регулировки гиперпараметров регуляризации выбранной модели результат немного улучшился. Но также возросло время обучения, в одном случае составив 34 минуты что значительно больше, чем у Random Forest
